In [21]:
import re
import os 
import datetime
from math import log
import csv
import pprint
import math
from collections import Counter
import numpy as np
import matplotlib.pyplot as plt
from pandas import Series, DataFrame
import pandas as pd
from pandas.io.parsers import read_csv
pd.set_option('max_colwidth',120)
from elasticsearch import Elasticsearch, helpers
from elasticsearch.helpers import scan
#from __future__ import division:q
import urllib

%matplotlib inline

pp = pprint.PrettyPrinter()

In [2]:
#bashcommand='pwd'

In [3]:
es = Elasticsearch([{'host':'atlas-kibana.mwt2.org', 'port':9200}],timeout=20)

In [4]:
list=['jobs_archive_2016-07-31','jobs_archive_2016-08-01','jobs_archive_2016-08-02','jobs_archive_2016-08-03','jobs_archive_2016-08-04','jobs_archive_2016-08-05','jobs_archive_2016-08-06','jobs_archive_2016-08-07','jobs_archive_2016-08-08','jobs_archive_2016-08-09','jobs_archive_2016-08-10','jobs_archive_2016-08-11','jobs_archive_2016-08-12','jobs_archive_2016-08-13','jobs_archive_2016-08-14','jobs_archive_2016-08-15','jobs_archive_2016-08-16','jobs_archive_2016-08-17','jobs_archive_2016-08-18','jobs_archive_2016-08-19','jobs_archive_2016-08-20','jobs_archive_2016-08-21','jobs_archive_2016-08-22','jobs_archive_2016-08-23','jobs_archive_2016-08-24','jobs_archive_2016-08-25','jobs_archive_2016-08-26','jobs_archive_2016-08-27','jobs_archive_2016-08-28','jobs_archive_2016-08-29','jobs_archive_2016-08-30','jobs_archive_2016-09-01']
#list=['jobs_archive_2016-07-31']
#print (list)
indices = es.cat.indices(index=list, h="index", request_timeout=10).split('\n')
indices = sorted(indices)
indices = [x.replace(" ", "") for x in indices if x != '']
 

In [5]:
print (indices)

['jobs_archive_2016-07-31', 'jobs_archive_2016-08-01', 'jobs_archive_2016-08-02', 'jobs_archive_2016-08-03', 'jobs_archive_2016-08-04', 'jobs_archive_2016-08-05', 'jobs_archive_2016-08-06', 'jobs_archive_2016-08-07', 'jobs_archive_2016-08-08', 'jobs_archive_2016-08-09', 'jobs_archive_2016-08-10', 'jobs_archive_2016-08-11', 'jobs_archive_2016-08-12', 'jobs_archive_2016-08-13', 'jobs_archive_2016-08-14', 'jobs_archive_2016-08-15', 'jobs_archive_2016-08-16', 'jobs_archive_2016-08-17', 'jobs_archive_2016-08-18', 'jobs_archive_2016-08-19', 'jobs_archive_2016-08-20', 'jobs_archive_2016-08-21', 'jobs_archive_2016-08-22', 'jobs_archive_2016-08-23', 'jobs_archive_2016-08-24', 'jobs_archive_2016-08-25', 'jobs_archive_2016-08-26', 'jobs_archive_2016-08-27', 'jobs_archive_2016-08-28', 'jobs_archive_2016-08-29', 'jobs_archive_2016-08-30', 'jobs_archive_2016-09-01']


In [6]:
ss={
 "query": { "wildcard": { "computingsite": "*IN2P3-CC*" } 
}, "_source": ["pandaid","batchid","computingsite","computingelement","corecount","creationtime","starttime","endtime",
   "cpuconsumptiontime","wall_time","modificationhost","maxrss","maxpss","jobstatus"]
#    ,"size": 5
    }



#ss= {"query": { "match_all": {} },"size": 10000} 
print (ss)



{'query': {'wildcard': {'computingsite': '*IN2P3-CC*'}}, '_source': ['pandaid', 'batchid', 'computingsite', 'computingelement', 'corecount', 'creationtime', 'starttime', 'endtime', 'cpuconsumptiontime', 'wall_time', 'modificationhost', 'maxrss', 'maxpss', 'jobstatus']}


In [7]:
#res = es.search(index='jobs_archive_2016-06-26', body=ss, filter_path=['hits.hits._source'],request_timeout=12000)

In [8]:
res=es.search(index=indices, body=ss,request_timeout=12000)
#print (res)
len(res)
#print (res)

4

In [9]:
 #res.get('hits').get('hits')

In [10]:
scroll=scan(es,index=indices, query=ss,scroll=u'5m', raise_on_error=True, preserve_order=False, size=500000,request_timeout=700000)

In [11]:
c=1
list2=[]
for name in scroll:
 a=name.get('_source')
 list2.append(a)
 c=c+1
print (c)
type(list2)
dfu = pd.DataFrame(list2)
 
#dfu.batchid.count()
dfu.head()
 
 

1086223


,batchid,computingelement,computingsite,corecount,cpuconsumptiontime,creationtime,endtime,jobstatus,maxpss,maxrss,modificationhost,pandaid,starttime,wall_time
0,12676829,ANALY_IN2P3-CC,ANALY_IN2P3-CC,1.0,2048,2016-07-30T21:02:57,2016-07-31T00:58:14,finished,2388987.0,2399236.0,ccwsge0830,2942443979,2016-07-31T00:16:04,2530
1,12676951,ANALY_IN2P3-CC,ANALY_IN2P3-CC,1.0,2055,2016-07-30T21:02:57,2016-07-31T00:57:02,finished,2399791.0,2410456.0,ccwsge0762,2942443977,2016-07-31T00:15:57,2465
2,12676827,ANALY_IN2P3-CC,ANALY_IN2P3-CC,1.0,2078,2016-07-30T21:02:58,2016-07-31T00:57:35,finished,2403533.0,2413992.0,ccwsge0760,2942444045,2016-07-31T00:17:30,2405
3,12676826,ANALY_IN2P3-CC,ANALY_IN2P3-CC,1.0,2096,2016-07-30T21:02:58,2016-07-31T00:57:42,finished,2408860.0,2419436.0,ccwsge1151,2942444048,2016-07-31T00:17:40,2402
4,12678265,ANALY_IN2P3-CC,ANALY_IN2P3-CC,NaN,1295,2016-07-31T00:41:32,2016-07-31T01:08:38,finished,1601431.0,1610764.0,ccwsge0713,2942609895,2016-07-31T00:41:56,1602


In [12]:
dfu.groupby(dfu.computingsite).computingsite.count()

computingsite
ANALY_IN2P3-CC          618671
IN2P3-CC                257901
IN2P3-CC-T3_MCORE           32
IN2P3-CC-T3_VM01         15633
IN2P3-CC-T3_VM02         35737
IN2P3-CC_MCORE           96169
IN2P3-CC_MCORE_HIMEM     55029
IN2P3-CC_VVL              7050
Name: computingsite, dtype: int64

In [13]:
dfu.groupby(dfu.computingsite).count()

,batchid,computingelement,corecount,cpuconsumptiontime,creationtime,endtime,jobstatus,maxpss,maxrss,modificationhost,pandaid,starttime,wall_time
computingsite,,,,,,,,,,,,,
ANALY_IN2P3-CC,601286,618481,585043,618671,618671,618671,618671,592553,592553,618671,618671,601244,618671
IN2P3-CC,193343,257801,257856,257901,257901,257901,257901,189065,189065,257901,257901,193408,257901
IN2P3-CC-T3_MCORE,0,32,32,32,32,32,32,0,0,32,32,0,32
IN2P3-CC-T3_VM01,15384,15583,15592,15633,15633,15633,15633,15331,15331,15633,15633,15386,15633
IN2P3-CC-T3_VM02,35064,35737,35737,35737,35737,35737,35737,35034,35034,35737,35737,35065,35737
IN2P3-CC_MCORE,91047,96169,96169,96169,96169,96169,96169,90868,90868,96169,96169,91049,96169
IN2P3-CC_MCORE_HIMEM,49738,55029,55029,55029,55029,55029,55029,49655,49655,55029,55029,49739,55029
IN2P3-CC_VVL,6914,7050,7050,7050,7050,7050,7050,6896,6896,7050,7050,6914,7050


In [14]:


#rint (res.get('hits').get('hits'))
#list.head()
#res[0]
#res['total']

In [15]:
dfu.head()

,batchid,computingelement,computingsite,corecount,cpuconsumptiontime,creationtime,endtime,jobstatus,maxpss,maxrss,modificationhost,pandaid,starttime,wall_time
0,12676829,ANALY_IN2P3-CC,ANALY_IN2P3-CC,1.0,2048,2016-07-30T21:02:57,2016-07-31T00:58:14,finished,2388987.0,2399236.0,ccwsge0830,2942443979,2016-07-31T00:16:04,2530
1,12676951,ANALY_IN2P3-CC,ANALY_IN2P3-CC,1.0,2055,2016-07-30T21:02:57,2016-07-31T00:57:02,finished,2399791.0,2410456.0,ccwsge0762,2942443977,2016-07-31T00:15:57,2465
2,12676827,ANALY_IN2P3-CC,ANALY_IN2P3-CC,1.0,2078,2016-07-30T21:02:58,2016-07-31T00:57:35,finished,2403533.0,2413992.0,ccwsge0760,2942444045,2016-07-31T00:17:30,2405
3,12676826,ANALY_IN2P3-CC,ANALY_IN2P3-CC,1.0,2096,2016-07-30T21:02:58,2016-07-31T00:57:42,finished,2408860.0,2419436.0,ccwsge1151,2942444048,2016-07-31T00:17:40,2402
4,12678265,ANALY_IN2P3-CC,ANALY_IN2P3-CC,NaN,1295,2016-07-31T00:41:32,2016-07-31T01:08:38,finished,1601431.0,1610764.0,ccwsge0713,2942609895,2016-07-31T00:41:56,1602


In [23]:
 dfu.groupby(['computingsite']).count()

,batchid,computingelement,corecount,cpuconsumptiontime,creationtime,endtime,jobstatus,maxpss,maxrss,modificationhost,pandaid,starttime,wall_time
computingsite,,,,,,,,,,,,,
ANALY_IN2P3-CC,601286,618481,585043,618671,618671,618671,618671,592553,592553,618671,618671,601244,618671
IN2P3-CC,193343,257801,257856,257901,257901,257901,257901,189065,189065,257901,257901,193408,257901
IN2P3-CC-T3_MCORE,0,32,32,32,32,32,32,0,0,32,32,0,32
IN2P3-CC-T3_VM01,15384,15583,15592,15633,15633,15633,15633,15331,15331,15633,15633,15386,15633
IN2P3-CC-T3_VM02,35064,35737,35737,35737,35737,35737,35737,35034,35034,35737,35737,35065,35737
IN2P3-CC_MCORE,91047,96169,96169,96169,96169,96169,96169,90868,90868,96169,96169,91049,96169
IN2P3-CC_MCORE_HIMEM,49738,55029,55029,55029,55029,55029,55029,49655,49655,55029,55029,49739,55029
IN2P3-CC_VVL,6914,7050,7050,7050,7050,7050,7050,6896,6896,7050,7050,6914,7050


In [17]:
#b=dfu.maxpss/1024/1024
#b.hist(bins=100)

In [18]:
#dfu.memory_usage()

In [24]:
dfu.to_csv('./ev_example_aug2016.csv')